In [189]:
import os
import numpy as np
print("numpy version: ", np.__version__)
import matplotlib.pyplot as plt
import librosa
import audiofile as af
import json
import soundfile as sf
import pandas as pd

numpy version:  1.26.4


In [190]:
print(np.__version__)

1.26.4


In [191]:
# go into each json file in the directory and extract the data into a df and create rows with each label at event level with id the id of the subject

# get the list of json files
def read_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json')]

    list_of_dicts = []
    

    # Loop through each JSON file and load its content into the dictionary
    for json_file in json_files:
        file_path = os.path.join(directory, json_file)
        with open(file_path, 'r', encoding='utf-8') as f:
            data_dict = {}
            data = json.load(f)
            data_dict[json_file] = data
            list_of_dicts.append(data_dict)

    return json_files,data_dict,list_of_dicts



In [192]:
# Specify your directory here
directory_path = 'train_classification_json'
_,_,train_json_list = read_json_files(directory_path)

In [193]:
# print(len(lala))
print(len(train_json_list))
print(train_json_list[5])

# works fine

1949
{'41283394_2.8_1_p4_2814.json': {'record_annotation': 'Normal', 'event_annotation': [{'start': '1405', 'end': '2683', 'type': 'Normal'}, {'start': '2827', 'end': '4142', 'type': 'Normal'}, {'start': '4250', 'end': '5728', 'type': 'Normal'}, {'start': '5894', 'end': '7153', 'type': 'Normal'}, {'start': '7686', 'end': '9075', 'type': 'Normal'}]}}


In [194]:
def gender_transform(gender):
    if gender=='0':
        gender = 'Male'
    else:
        gender = 'Female'
    return gender

def location_transform(location):
    if location =='p1':
        return 'left posterior'
    elif location =='p2':
        return 'left lateral'
    elif location =='p3':
        return 'right posterior'
    else:
        return 'right lateral'

In [195]:
def record_level_df_create(list):

    # List to collect rows for DataFrame
    rows = []

    # Access and print the 'record_annotation' for each dictionary in the list
    for dictionary in list:
        for file_name, data in dictionary.items():
            parts = file_name.split('_')
            # Add a row with the filename and record_annotation
            rows.append({
                # the patient number is the file name until the first _
                'recording_number':parts[4].split('.')[0],
                'patient_number': parts[0],
                'age':float(parts[1]),
                'gender':gender_transform(parts[2]),
                'recording_location':location_transform(parts[3]),
                'record_annotation': data.get('record_annotation', 'N/A')
            })

    record_level_df = pd.DataFrame(rows)
    record_level_df.set_index('recording_number', inplace=True)
    return record_level_df

train_record_level_df = record_level_df_create(train_json_list)
# count the number of males and females
print(train_record_level_df['record_annotation'].value_counts())

train_record_level_df

record_annotation
Normal          1303
DAS              248
Poor Quality     177
CAS              126
CAS & DAS         95
Name: count, dtype: int64


,patient_number,age,gender,recording_location,record_annotation
recording_number,,,,,
1340,42272778,7.5,Male,left posterior,Normal
2306,41166273,3.9,Male,right posterior,Normal
2178,41275381,2.6,Male,right lateral,Normal
3326,65091224,2.9,Male,right lateral,Normal
2410,65091359,10.4,Male,left lateral,Normal
...,...,...,...,...,...
2754,41267024,0.3,Male,right lateral,Normal
245,41260684,3.8,Female,left posterior,Normal
2022,41074892,10.0,Female,left lateral,Normal


In [196]:
def event_level_df_create(list):

    # List to collect rows for DataFrame
    rows = []
    # Access and print the 'record_annotation' for each dictionary in the list
    for dictionary in list:
        for file_name, data in dictionary.items():
            parts = file_name.split('_')
            # Add a row with the filename and record_annotation
            if not data.get('event_annotation', []):
                rows.append({
                    # the patient number is the file name until the first _
                    'recording_number':parts[4].split('.')[0],
                    'patient_number': parts[0],
                    'age':float(parts[1]),
                    'gender':gender_transform(parts[2]),
                    'recording_location':location_transform(parts[3]),
                    'start': 'N/A',
                    'end': 'N/A',
                    'event_annotation': data.get('event_annotation', 'N/A')
                })
            else:
                for event in data.get('event_annotation', []):
                    rows.append({
                        # the patient number is the file name until the first _
                        'recording_number':parts[4].split('.')[0],
                        'patient_number': parts[0],
                        'age':float(parts[1]),
                        'gender':gender_transform(parts[2]),
                        'recording_location':location_transform(parts[3]),
                        'start': event.get('start', 'N/A'),
                        'end': event.get('end', 'N/A'),
                        'event_annotation': event.get('type', 'N/A'),
                    })

    event_level_df = pd.DataFrame(rows)
    event_level_df.set_index('recording_number', inplace=True)

    return event_level_df


train_event_level_df = event_level_df_create(train_json_list)


train_event_level_df

,patient_number,age,gender,recording_location,start,end,event_annotation
recording_number,,,,,,,
1340,42272778,7.5,Male,left posterior,6422,7457,Normal
1340,42272778,7.5,Male,left posterior,8436,9169,Normal
1340,42272778,7.5,Male,left posterior,4409,5786,Normal
1340,42272778,7.5,Male,left posterior,79,4164,Normal
2306,41166273,3.9,Male,right posterior,796,2069,Normal
...,...,...,...,...,...,...,...
3115,41161556,1.7,Male,left posterior,6672,7176,Fine Crackle
3115,41161556,1.7,Male,left posterior,7991,8615,Fine Crackle
97,41112999,3.2,Male,left posterior,3776,5043,Normal


In [197]:
train_event_level_df['event_annotation'].value_counts()

event_annotation
Normal            5159
Fine Crackle       912
Wheeze             452
[]                 177
Coarse Crackle      49
Rhonchi             39
Wheeze+Crackle      30
Stridor             15
Name: count, dtype: int64

In [198]:
# Specify your directory here
directory_path = 'test_classification_json/2022/inter_test_json'
_,_,inter_test_json_list = read_json_files(directory_path)

In [199]:
inter_test_record_level_df = record_level_df_create(inter_test_json_list)
inter_test_record_level_df

,patient_number,age,gender,recording_location,record_annotation
recording_number,,,,,
4147,41209060,4.4,Female,right posterior,Normal
3576,65109516,0.2,Male,right posterior,DAS
3579,64714172,3.6,Female,right lateral,Normal
4166,65118898,0.7,Male,right lateral,CAS
3581,64714172,3.6,Female,left lateral,Normal
...,...,...,...,...,...
4110,65121853,1.5,Male,right lateral,DAS
1146,40888395,3.4,Male,left posterior,Normal
3624,65071184,4.8,Male,left lateral,Normal


In [200]:
inter_test_record_level_df['record_annotation'].value_counts()

record_annotation
Normal          241
CAS              65
DAS              24
CAS & DAS        17
Poor Quality      8
Name: count, dtype: int64

In [201]:
inter_test_event_level_df = event_level_df_create(inter_test_json_list)
inter_test_event_level_df

,patient_number,age,gender,recording_location,start,end,event_annotation
recording_number,,,,,,,
4147,41209060,4.4,Female,right posterior,81,2532,Normal
4147,41209060,4.4,Female,right posterior,4773,6877,Normal
4147,41209060,4.4,Female,right posterior,6934,9383,Normal
4147,41209060,4.4,Female,right posterior,9426,11178,Normal
4147,41209060,4.4,Female,right posterior,11254,13551,Normal
...,...,...,...,...,...,...,...
1146,40888395,3.4,Male,left posterior,8025,8803,Normal
3624,65071184,4.8,Male,left lateral,2637,4916,Normal
3664,65109516,0.2,Male,right lateral,2059,3319,Normal


In [202]:
inter_test_event_level_df['event_annotation'].value_counts()

event_annotation
Normal            1040
Wheeze             305
Fine Crackle        80
[]                   8
Coarse Crackle       3
Wheeze+Crackle       1
Name: count, dtype: int64

In [203]:
# Specify your directory here
directory_path = 'test_classification_json/2022/intra_test_json'
_,_,intra_test_json_list = read_json_files(directory_path)

In [204]:
intra_test_record_level_df = record_level_df_create(intra_test_json_list)
intra_test_record_level_df

,patient_number,age,gender,recording_location,record_annotation
recording_number,,,,,
2793,65045423,5.2,Female,right lateral,DAS
2915,41281333,0.2,Female,left lateral,CAS & DAS
1280,63613821,9.2,Female,left lateral,DAS
2927,41279835,14.1,Male,left lateral,DAS
190,41159145,3.3,Male,left lateral,Normal
...,...,...,...,...,...
260,65044049,0.9,Female,right lateral,DAS
2624,64743918,7.0,Male,right posterior,CAS
1095,65064302,3.2,Female,right posterior,Normal


In [205]:
intra_test_record_level_df['record_annotation'].value_counts()

record_annotation
Normal          241
DAS              75
CAS              42
CAS & DAS        19
Poor Quality      2
Name: count, dtype: int64

In [206]:
intra_test_event_level_df = event_level_df_create(intra_test_json_list)
intra_test_event_level_df

,patient_number,age,gender,recording_location,start,end,event_annotation
recording_number,,,,,,,
2793,65045423,5.2,Female,right lateral,5547,6020,Coarse Crackle
2793,65045423,5.2,Female,right lateral,6842,7464,Coarse Crackle
2793,65045423,5.2,Female,right lateral,3578,4531,Coarse Crackle
2915,41281333,0.2,Female,left lateral,7269,7686,Wheeze
2915,41281333,0.2,Female,left lateral,7755,8400,Fine Crackle
...,...,...,...,...,...,...,...
898,40801342,4.0,Female,left lateral,3463,4239,Fine Crackle
898,40801342,4.0,Female,left lateral,6101,6957,Fine Crackle
898,40801342,4.0,Female,left lateral,2108,2830,Fine Crackle


In [207]:
intra_test_event_level_df['event_annotation'].value_counts()

event_annotation
Normal            688
Fine Crackle      175
Wheeze            108
Coarse Crackle     14
Rhonchi            14
Wheeze+Crackle      3
Stridor             2
[]                  2
Name: count, dtype: int64

In [210]:
train_record_level_df

,patient_number,age,gender,recording_location,record_annotation
recording_number,,,,,
1340,42272778,7.5,Male,left posterior,Normal
2306,41166273,3.9,Male,right posterior,Normal
2178,41275381,2.6,Male,right lateral,Normal
3326,65091224,2.9,Male,right lateral,Normal
2410,65091359,10.4,Male,left lateral,Normal
...,...,...,...,...,...
2754,41267024,0.3,Male,right lateral,Normal
245,41260684,3.8,Female,left posterior,Normal
2022,41074892,10.0,Female,left lateral,Normal


In [211]:
train_event_level_df

,patient_number,age,gender,recording_location,start,end,event_annotation
recording_number,,,,,,,
1340,42272778,7.5,Male,left posterior,6422,7457,Normal
1340,42272778,7.5,Male,left posterior,8436,9169,Normal
1340,42272778,7.5,Male,left posterior,4409,5786,Normal
1340,42272778,7.5,Male,left posterior,79,4164,Normal
2306,41166273,3.9,Male,right posterior,796,2069,Normal
...,...,...,...,...,...,...,...
3115,41161556,1.7,Male,left posterior,6672,7176,Fine Crackle
3115,41161556,1.7,Male,left posterior,7991,8615,Fine Crackle
97,41112999,3.2,Male,left posterior,3776,5043,Normal


In [212]:
# count how many are bad quality
train_record_level_df['record_annotation'].value_counts()

record_annotation
Normal          1303
DAS              248
Poor Quality     177
CAS              126
CAS & DAS         95
Name: count, dtype: int64

In [213]:
# def extract_mfccs(wav_files, n_mfcc=13):
#     mfccs_dict = {}
    
#     for wav_file in wav_files:
#         # Load the audio file
#         y, sr = librosa.load(wav_file, sr=None)
        
#         # Compute MFCCs
#         mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
#         # Store the MFCCs in the dictionary
#         mfccs_dict[wav_file] = mfccs
    
#     return mfccs_dict

In [214]:
train_mfccs_dict = extract_mfccs(train_wav_list)
test_mfccs_dict = extract_mfccs(test_wav_list)

In [215]:
# mean_mfccs =  [np.mean(train_mfccs_dict[train_wav_list[i]],axis=1) for i in range(len(train_wav_list))]

# stddev_mfccs = [np.std(train_mfccs_dict[train_wav_list[i]],axis=1) for i in range(len(train_wav_list))]

In [216]:
# i=0
# for file, mfccs in train_mfccs_dict.items():
#     if i ==1948:
#         print(f"MFCCs for {file}:")
#         print(np.mean(mfccs,axis=1))
#     i+=1
        

In [217]:
# Create a DataFrame from the mean MFCCs and with a column for the filename

# train_mfccs_df = pd.DataFrame(mean_mfccs, columns=[f'mfcc_{i}_mean' for i in range(1, 14)])
# train_mfccs_df

In [218]:
# train_mfccs_df_1=pd.concat([train_mfccs_df,pd.DataFrame(stddev_mfccs, columns=[f'mfcc_{i}_stddev' for i in range(1, 14)])],axis=1)
# train_mfccs_df_1

In [219]:
# test_df= train_record_level_df.reset_index()

# # Reset indices of both DataFrames (if necessary)
# test_df.reset_index(drop=True, inplace=True)
# train_mfccs_df_1.reset_index(drop=True, inplace=True)

# concatenated_train_df = pd.concat([test_df, train_mfccs_df_1], axis=1)
# concatenated_train_df